# 投资者情绪专题

## 数据准备

In [1]:
import sys
import os
import cudf  #CUDA计算
import pandas as pd

# 自行编写的包
sys.path.append('/home/ubuntu/notebook/Investor-Sentiment')
sys.path.append('/usr/local/stata17/utilities')

# Stata
from pystata import config  # noqa

config.init('mp')
# ------------------------------数据集路径----------------------------------#
DATASETS_PATH = '/data/DataSets/investor_sentiment/'


  ___  ____  ____  ____  ____ ©
 /__    /   ____/   /   ____/      17.0
___/   /   /___/   /   /___/       MP—Parallel Edition

 Statistics and Data Science       Copyright 1985-2021 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-STATA-PC        https://www.stata.com
                                   979-696-4600        stata@stata.com

Stata license: Single-user 8-core , expiring  1 Jan 2025
Serial number: 501709301094
  Licensed to: Colin's Stata
               Love U

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. More than 2 billion observations are allowed; see help obs_advice.
      3. Maximum number of variables is set to 5,000; see help set_maxvar.


In [2]:
df = (
        cudf.read_parquet(
                DATASETS_PATH + 'FORUM_SENT_TRANS.parquet',
                columns=['PostDate', 'Stockcode', 'PostSource', 'PositiveSentIndexA', 'PositiveSentIndexB', 'TotalPosts', 'AvgReadings',
                         'AvgComments', 'AvgPositThumbUps'])
        .query("PostDate>=20131231 ")

)

In [3]:
df.set_index(['PostDate', 'Stockcode']).sort_index().to_pandas()

PostSource  PositiveSentIndexA  PositiveSentIndexB  \
PostDate Stockcode                                                       
20131231 000030.SZ           1                 NaN                 NaN   
         000030.SZ           1                 NaN                 NaN   
         000034.SZ           1                 NaN                 NaN   
         000034.SZ           1                 NaN                 NaN   
         000043.SZ           1                 NaN                 NaN   
...                        ...                 ...                 ...   
20211016 900948.SH           1                 NaN                 NaN   
         900952.SH           1                 NaN                 NaN   
         900953.SH           1                 NaN                 NaN   
         900955.SH           1                 NaN                 NaN   
         900957.SH           1                 NaN                 NaN   

                    TotalPosts  AvgReadings  AvgComments  AvgPositThumbUps  
PostDate Stockcode                                                          
20131231 000030.SZ           0          NaN          NaN               NaN  
         000030.SZ           0          NaN          NaN               NaN  
         000034.SZ           0          NaN          NaN               NaN  
         000034.SZ           0          NaN          NaN               NaN  
         000043.SZ           0          NaN          NaN               NaN  
...                        ...          ...          ...               ...  
20211016 900948.SH           0          NaN          NaN               NaN  
         900952.SH           0          NaN          NaN               NaN  
         900953.SH           0          NaN          NaN               NaN  
         900955.SH           0          NaN          NaN               NaN  
         900957.SH           0          NaN          NaN               NaN  

[11846404 rows x 7 columns]

## 指标计算

计算加权影响力,参数为影响力系数,点赞评论阅读

In [4]:
# 每日总量
df['DayTotalPosts'] = df.groupby(['PostDate', 'Stockcode'])['TotalPosts'].transform('sum')
df['DayAvgReadings'] = df.groupby(['PostDate', 'Stockcode'])['AvgReadings'].transform('sum')
df['DayAvgComments'] = df.groupby(['PostDate', 'Stockcode'])['AvgComments'].transform('sum')
df['DayAvgPositThumbUps'] = df.groupby(['PostDate', 'Stockcode'])['AvgPositThumbUps'].transform('sum')

# 加权因子总量
df['DayTotalPostsFactor'] = df['TotalPosts'] / df['DayTotalPosts']
df['DayAvgReadingsFactor'] = df['AvgReadings'] / df['DayAvgReadings']
df['DayAvgCommentsFactor'] = df['AvgComments'] / df['DayAvgComments']
df['DayAvgPositThumbUpsFactor'] = df['AvgPositThumbUps'] / df['DayAvgPositThumbUps']

# 个股加权情绪因子
df['PositiveSentIndexAFactor'] = (
        (df['DayTotalPostsFactor'] * 0.15 + df['DayAvgReadingsFactor'] * 0.5 +
         df['DayAvgCommentsFactor'] * 0.2 + df['DayAvgPositThumbUpsFactor'] * 0.15)
        * df['PositiveSentIndexA']
)

df['PositiveSentIndexBFactor'] = (
        (df['DayTotalPostsFactor'] * 0.15 + df['DayAvgReadingsFactor'] * 0.5 +
         df['DayAvgCommentsFactor'] * 0.2 + df['DayAvgPositThumbUpsFactor'] * 0.15)
        * df['PositiveSentIndexB']
)

# 每日情绪因子
df['DayPositiveSentIndexA'] = df.groupby(['PostDate', 'Stockcode'])['PositiveSentIndexAFactor'].transform('sum')
df['DayPositiveSentIndexB'] = df.groupby(['PostDate', 'Stockcode'])['PositiveSentIndexBFactor'].transform('sum')

# 转换为时间序列数据
df = df.groupby(['PostDate', ]).first()[
        ['DayPositiveSentIndexA', 'DayPositiveSentIndexB',
         'DayTotalPosts', 'DayAvgReadings', 'DayAvgComments', 'DayAvgPositThumbUps']].fillna(0)

df.to_pandas()

,DayPositiveSentIndexA,DayPositiveSentIndexB,DayTotalPosts,DayAvgReadings,DayAvgComments,DayAvgPositThumbUps
PostDate,,,,,,
20131231,0.000000,0.000000,0,0.00,0.00,0.0
20140101,0.000000,0.000000,0,0.00,0.00,0.0
20140102,0.000000,0.000000,0,0.00,0.00,0.0
20140103,0.000000,0.000000,0,203374.00,674.00,0.0
20140104,0.000000,0.000000,0,0.00,0.00,0.0
...,...,...,...,...,...,...
20211012,0.000000,0.000000,11,145.91,1.27,0.0
20211013,-0.600000,-1.098612,10,189.60,1.20,4.0
20211014,-0.500000,-0.847298,8,158.13,2.00,0.5
